In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import h5py

In [ ]:
gpu_devices = tf.config.list_physical_devices('GPU')
for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
with h5py.File("/content/drive/MyDrive/database/nDataset1.h5", 'r') as h5:
    X = h5['X_train'][:]
    y = h5['y_train'][:]
print(X.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=78)
print(X_train.shape)
print(X_test.shape)

In [ ]:
X = None
y = None

In [ ]:
utils = tf.keras.utils
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

In [ ]:
def create_model():
    np.random.seed(1233)
    num_classes = 2
    #base_model = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(160, 160,3))
    base_model = keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(160, 160,3))
    for layer in base_model.layers:
        layer.trainable = False
    cnn = keras.models.Sequential()
    cnn.add(base_model)
    cnn.add(keras.layers.Flatten())
    model = keras.models.Sequential()
    timedistriLay = keras.layers.TimeDistributed(cnn,  input_shape=(30, 160, 160, 3))
    model.add(timedistriLay)
    model.add(keras.layers.LSTM(30 , return_sequences= True))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(90)))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.GlobalAveragePooling1D())
    #model.add(keras.layers.Dense(1024, activation='gelu'))
    #model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(512, activation='gelu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(num_classes, activation="sigmoid"))
    adam = keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    #model.load_weights('/content/drive/MyDrive/Trained Model/VGG19_3.hdf5')
    rms = keras.optimizers.RMSprop()
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])
    return model

In [ ]:
t_model = create_model()

In [ ]:
t_model.summary()

In [ ]:
keras.utils.plot_model(t_model, show_shapes=True)

In [ ]:
callbacks = tf.keras.callbacks

In [ ]:
class AccuracyHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []
        self.loss = []
        self.val_loss = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

history = AccuracyHistory()
earlyStopping = callbacks.EarlyStopping(monitor='val_loss', patience=8,min_delta=1e-5, verbose=0, mode='min')
mcp_save = callbacks.ModelCheckpoint('VGG19_1.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='val_loss',patience=1, verbose=2,factor=0.5,min_lr=0.00000001)

In [ ]:
batch_size = 3
epochs = 5

In [ ]:
import time
millis = int(round(time.time() * 1000))
print("started at " , millis)

t_model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),callbacks=[earlyStopping, mcp_save, reduce_lr_loss, history])

In [ ]:
t_model.save_weights('/content/drive/MyDrive/Trained Model/VGG19_4.hdf5')

In [ ]:
import matplotlib.pyplot as plt
acc = history.acc
val_acc = history.val_acc
loss = history.loss
val_loss = history.val_loss
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()